# Sparse Autoencoder Training Demo

## Setup

In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from sparse_autoencoder import TensorActivationStore, SparseAutoencoder
from sparse_autoencoder.source_data.pile_uncopyrighted import PileUncopyrightedDataset
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
from transformers import GPT2TokenizerFast
import torch
import wandb

In [3]:
print(torch.__version__)
device = get_device()

1.13.1+cu117


### Source Dataset

In [4]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
source_data = PileUncopyrightedDataset(tokenizer=tokenizer)

(…)gingface.co/gpt2/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)face.co/gpt2/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

(…)ingface.co/gpt2/resolve/main/config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

### Source Model

In [ ]:
src_model = HookedTransformer.from_pretrained("solu-1l", dtype="float32")
src_d_mlp = src_model.cfg.d_mlp
src_d_mlp

Loaded pretrained model solu-1l into HookedTransformer


2048

### Activation Store

In [6]:
max_items = 2_000_000
store = TensorActivationStore(max_items, src_d_mlp, device)

### Autoencoder

In [10]:
autoencoder = SparseAutoencoder(src_d_mlp, src_d_mlp * 8, torch.zeros(src_d_mlp))
autoencoder

SparseAutoencoder(
  (encoder): Sequential(
    (0): TiedBias(position=TiedBiasPosition.PRE_ENCODER)
    (1): ConstrainedUnitNormLinear(in_features=2048, out_features=16384, bias=True)
    (2): ReLU()
  )
  (decoder): Sequential(
    (0): ConstrainedUnitNormLinear(in_features=16384, out_features=2048, bias=False)
    (1): TiedBias(position=TiedBiasPosition.POST_DECODER)
  )
)

## Training

If you initialise [wandb](https://wandb.ai/site), the pipeline will automatically log all metrics to wandb.

In [8]:
# wandb.init(project="sparse-autoencoder", dir=".cache/wandb")

In [11]:
# pipeline(
#     src_model=src_model,
#     src_model_activation_hook_point="blocks.0.mlp.hook_post",
#     src_model_activation_layer=0,
#     src_dataloader=src_dataloader,
#     activation_store=store,
#     num_activations_before_training=max_items,
#     autoencoder=autoencoder,
#     device=device,
# )